In [1]:
from aequitas.flow.utils.colab import get_examples
from aequitas.flow.experiment import Experiment
from pathlib import Path
from omegaconf import DictConfig
from aequitas.flow.utils.logging import clean_handlers
from aequitas.flow.utils.results import read_results

# [Train a Model with Fairness Considerations](https://colab.research.google.com/github/dssg/aequitas/blob/notebooks/aequitas_flow_experiment.ipynb)

In [2]:
lightgbm_config = {
    'lightgbm': {
        'classpath': 'aequitas.flow.methods.base_estimator.lightgbm.LightGBM',
        'args': {
            'boosting_type': ['dart', 'gbdt'],
            'enable_bundle': [False,],
            'n_estimators': {'type': 'int', 'range': [10, 100]},
            'min_child_samples': {'type': 'int', 'range': [1, 500], 'log': True},
            'learning_rate': {'type': 'float', 'range': [0.001, 0.1]},
            'random_state': [42,],
        }
    }
}

fairgbm_config = {
    'fairgbm': {
        'classpath': 'aequitas.flow.methods.inprocessing.fairgbm.FairGBM',
        'args': {
            'global_constraint_type': ['FPR','FNR'],
            'global_target_fpr': [0.05],
            'global_target_fnr': {'type': 'float', 'range': [0.4, 0.6]},
            'constraint_type': ['fpr'],
            'constraint_fpr_threshold': [0],
            'proxy_margin': [1],
            'multiplier_learning_rate': {'type': 'float', 'range': [0.00001, 1.0], 'log': True},
            'constraint_stepwise_proxy': 'cross_entropy',
            'boosting_type': ['dart', 'gbdt'],
            'enable_bundle': [False,],
            'n_estimators': {'type': 'int', 'range': [10, 100]},
            'min_child_samples': {'type': 'int', 'range': [1, 500], 'log': True},
            'learning_rate': {'type': 'float', 'range': [0.001, 0.1]},
            'random_state': [42,]
        }
    }
}

sampling_config = {
    'sampling': {
        'classpath': 'aequitas.flow.methods.preprocessing.PrevalenceSampling',
        'args': {}
    }
}

random_forest_config = {
    'random_forest': {
        'classpath': 'aequitas.flow.methods.base_estimator.random_forest.RandomForest',
        'args': {
            'n_estimators': {'type': 'int', 'range': [10, 100]},
            'min_samples_leaf': {'type': 'int', 'range': [1, 500], 'log': True},
            'random_state': [42,]
        }
    }
}

group_threshold_config = {
    'group_threshold': {
        'classpath': 'aequitas.flow.methods.postprocessing.GroupThreshold',
        'args': {
            'threshold_type': 'fpr',
            'threshold_value': 0.05,
        }
    }
}

logistic_regression_config = {
    'logistic_regression': {
        'classpath': 'aequitas.flow.methods.base_estimator.logistic_regression.LogisticRegression',
        'args': {
            'penalty': ['l2', None],
            'tol': {'type': 'float', 'range': [1e-5, 1], 'log': True},
            'C': {'type': 'float', 'range': [1e-5, 1e2], 'log': True},
            'random_state': [42, ]
        }
    }
}

In [3]:
composed_config_lgbm = {'lightgbm':
  {
    'inprocessing': lightgbm_config,
    'type': 'base estimator',
  },
}

composed_config_fairgbm = {'fairgbm':
  {
    'inprocessing': fairgbm_config,
    'type': 'in-processing',
  },
}

composed_config_random_forest = {'Random Forest + Undersampling':
  {
    'preprocessing': sampling_config,
    'inprocessing': random_forest_config,
    'type': 'pre-processing',
  },
}

composed_config_logistic_regression = {'Logistic Regression':
  {
    'inprocessing': logistic_regression_config,
    'postprocessing': group_threshold_config,
    'type': 'post-processing',
  },
}

In [4]:
baf_sample_config = {
    'baf_sample': {
        'classpath': 'aequitas.flow.datasets.BankAccountFraud',
        'threshold': {
            'threshold_type': 'fpr',
            'threshold_value': 0.05,
        },
        'args': {'variant': 'Sample'},
    }
}

In [5]:
config = {
  'optimization': {
      'n_trials': 50,  # Number of runs per algorithm.
      'n_jobs': 1,
      'sampler': 'RandomSampler',  # The sampler for hyperparameters.
      'sampler_args': {'seed': 42},
  },
  'datasets': [baf_sample_config],
  'methods': [composed_config_lgbm,  composed_config_logistic_regression, composed_config_random_forest],
}

In [6]:
get_examples("configs")

[INFO] 2024-02-11 20:40:01 utils.colab - Downloading examples from fairflow repository.


INFO:utils.colab:Downloading examples from fairflow repository.


[INFO] 2024-02-11 20:48:05 utils.colab - Examples downloaded.


INFO:utils.colab:Examples downloaded.


In [6]:
clean_handlers()

In [7]:
experiment = Experiment(config=DictConfig(config), name="baf_exp")

experiment.run()

[INFO] 2024-02-11 23:14:27 Experiment - Instantiating Experiment class.
[INFO] 2024-02-11 23:14:27 Experiment - Beginning Experiment.
[INFO] 2024-02-11 23:14:27 Experiment - Saving objects to 'C:\Users\15783\source\repos\Ethical-Framework\Aequitas\demo\artifacts\baf_exp'.
[INFO] 2024-02-11 23:14:27 datasets.BankAccountFraud - Instantiating a BankAccountFraud dataset.
[INFO] 2024-02-11 23:14:27 datasets.BankAccountFraud - Loading data from C:\Users\15783\source\repos\Ethical-Framework\Aequitas\.venv\Lib\site-packages\aequitas\datasets\BankAccountFraud\Sample.parquet


[INFO] 2024-02-11 23:14:27 Experiment - Using 'baf_sample'.
[INFO] 2024-02-11 23:14:27 Experiment - Testing 'lightgbm', saved in 'C:\Users\15783\source\repos\Ethical-Framework\Aequitas\demo\artifacts\baf_exp\baf_sample'.
[INFO] 2024-02-11 23:14:30 methods.postprocessing.Threshold - Instantiating postprocessing Threshold.


[I 2024-02-11 23:14:30,137] A new study created in memory with name: no-name-37b9f5e7-de79-44c1-8a79-5152f2962683


[INFO] 2024-02-11 23:14:30 methods.base_estimator.LightGBM - Instantiating LightGBM model.
[INFO] 2024-02-11 23:14:30 methods.base_estimator.LightGBM - Fitting LightGBM model.


c:\Users\15783\source\repos\Ethical-Framework\Aequitas\.venv\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\15783\source\repos\Ethical-Framework\Aequitas\.venv\Lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


...

INFO:methods.base_estimator.RandomForest:Finished predicting with RandomForest model.


[INFO] 2024-02-11 23:20:15 methods.postprocessing.Threshold - Transforming predictions.


INFO:methods.postprocessing.Threshold:Transforming predictions.


[INFO] 2024-02-11 23:20:15 methods.postprocessing.Threshold - Finished transforming predictions.


INFO:methods.postprocessing.Threshold:Finished transforming predictions.
[I 2024-02-11 23:20:16,025] Trial 49 finished with value: 0.4603174603174603 and parameters: {'learner_type': 'algorithm', 'learner_algorithm_0_n_estimators': 57, 'learner_algorithm_0_min_samples_leaf': 104, 'learner_algorithm_0_random_state': 42}. Best is trial 4 with value: 0.5978835978835979.


It seems that FairGBM may be incompatible in Window environemnt, which cause an error when executing the experiment with FairGBM included. Current experiment result is excluding the FairGBM methods. May repeat the experiment in Linux Server environment with FairGBM included.

In [8]:
results = read_results(Path("artifacts/baf_exp"))

IndexError: list index out of range

In [ ]:
dataset_results = results["baf_sample"]

for method, method_results in dataset_results.items():
    filtered_method_results = []
    for iteration in method_results:
        if 0.03 < iteration.validation_results["fpr"] < 0.07:  # Here we implement the slack
            filtered_method_results.append(iteration)
    dataset_results[method] = filtered_method_results

results["baf_sample"] = dataset_results

In [ ]:
from aequitas.flow.plots.pareto import Plot

plot = Plot(
    results,
    "baf_sample",
    "Predictive Equality",
    "TPR",
    split="validation"
)

plot.visualize()

In [ ]:
dataset = BankAccountFraud("Sample")
dataset._download = False
dataset.load_data()
dataset.create_splits()

plot.bias_audit(119, dataset.test, "customer_age_bin", metrics=["fpr"], results_path="artifacts/baf_exp")

In [ ]:
plot.disparities(119, dataset.test, "customer_age_bin", metrics=["fpr"], results_path="artifacts/baf_exp")